In [1]:
# setting device on GPU if available, else CPU
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
print()

#Additional Info when using cuda
if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_reserved(0)/1024**3,1), 'GB')


Using device: cuda

NVIDIA GeForce RTX 2080 Ti
Memory Usage:
Allocated: 0.0 GB
Cached:    0.0 GB


In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [3]:
import torch
import numpy as np
import os
import torch.nn as nn
from tqdm import tqdm
import json
from functools import partial
from torch import einsum, nn
import torch.nn.functional as F

In [4]:
def findAllFile(base):
    file_path = []
    for root, ds, fs in os.walk(base, followlinks=True):
        for f in fs:
            fullname = os.path.join(root, f)
            file_path.append(fullname)
    return file_path



In [5]:
from utils.motion_processing.hml_process import recover_from_ric, recover_root_rot_pos,recover_from_rot
import utils.vis_utils.plot_3d_global as plot_3d
import matplotlib.pyplot as plt
def vis(mot , dset , name = "motion", joints = 22 , zero_trans = False):
    mot = (
                    dset.datasets[0]
                    .inv_transform(mot.cpu())
                    .squeeze()
                    .float()
                )



    xyz = np.array(recover_from_ric(mot , joints))

    
    plot_3d.render(xyz , f"/srv/hays-lab/scratch/sanisetty3/music_motion/ACTMG/render/{name}.gif")

In [6]:
from configs.config_t2m import cfg, get_cfg_defaults

cfg = get_cfg_defaults()
cfg.merge_from_file("/srv/hays-lab/scratch/sanisetty3/music_motion/ATCMG/checkpoints/motion_translation/motion_translation.yaml")

In [7]:
from core.datasets.conditioner import ConditionProvider, ConditionFuser
from core.datasets.multimode_dataset import MotionAudioTextDataset, load_dataset, simple_collate

/srv/hays-lab/flash5/sanisetty3/miniconda3/envs/tgm3d/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
from core.models.attend import Attention
from core import AttentionParams
from core import AttentionParams, TranslationTransformerParams, PositionalEmbeddingParams, PositionalEmbeddingType, MotionRep, AudioRep, TextRep


In [9]:
# from core.models.generation.translation_transformer import TranslationTransformer
# translation_tranformer = TranslationTransformer(cfg.translation_transformer).cuda()

In [10]:
# condition_provider.motion_max_length

In [11]:
dataset_args = cfg.dataset


In [12]:
condition_provider = ConditionProvider(
            motion_rep=MotionRep(dataset_args.motion_rep),
            audio_rep=AudioRep(dataset_args.audio_rep),
            text_rep=TextRep(dataset_args.text_rep),
            motion_padding=dataset_args.motion_padding,
            audio_padding=dataset_args.audio_padding,
            motion_max_length_s=10,
            audio_max_length_s=10,
        )

In [21]:
from core.datasets.multimode_dataset import MotionAudioTextDataset
from utils.motion_processing.skeleton import Skeleton, t2m_kinematic_chain , body_joints_id, t2m_raw_body_offsets

In [22]:
dset = MotionAudioTextDataset("moyo" , "/srv/hays-lab/scratch/sanisetty3/motionx" ,motion_rep = "body" , hml_rep = "gprvc", split = "test"   )

Total number of motions moyo: 9 and texts 9


In [23]:
sample = next(iter(dset))

In [25]:
sample["motion"].positions.shape

(251, 63)

In [26]:
new_mot = sample["motion"][10:100]

In [30]:
new_mot[:100] = sample["motion"][:100]

ValueError: setting an array element with a sequence. The requested array would exceed the maximum number of dimension of 2.

In [24]:
len(sample["motion"])

251

In [126]:
print(sample["motion"].root_params.shape)

AttributeError: 'NoneType' object has no attribute 'shape'

In [23]:
print(sample["motion"].positions.shape)

(251, 45)


In [24]:
print(sample["motion"].rotations.shape)

(251, 90)


In [25]:
print(sample["motion"].velocity.shape)


(251, 45)


In [141]:
print(sample["motion"].contact.shape)

AttributeError: 'NoneType' object has no attribute 'shape'

In [ ]:
train_loader = torch.utils.data.DataLoader(
        dset,
        4,
        sampler=sampler,
        collate_fn=partial(simple_collate , conditioner = condition_provider),
        drop_last=True,
    )

In [ ]:
for inputs , conditions in train_loader:
    break
    

In [79]:
conditions["text"][0].shape

torch.Size([4, 23, 768])

In [47]:
txts = list(inputs["texts"])

In [37]:
inputs["names"]

array(['kungfu/subset_0003/Shaolin_Kung_Fu_Wushu_Basic_Tiger_Sword_Training_clip_34',
       'kungfu/subset_0001/Dragon_Sword_-_Shaolin_Kung_Fu_Wushu_Da_Mo_Sword_clip_15',
       'kungfu/subset_0000/Boxing_Kicking_Jumping_clip_7',
       'kungfu/subset_0002/Kung_Fu_Wushu_Butterfly_Kick_Bkick_clip_49'],
      dtype='<U75')

In [16]:
motion = inputs["motion"][0]

In [19]:
motion_ = motion[... , [0,1,2,3,-4,-3,-2,-1]]

In [24]:
pred = torch.rand_like(motion_)

In [27]:
torch.nn.functional.mse_loss(motion_[...,:4] , pred[...,:4])

tensor(3.3602, device='cuda:0')

In [29]:
torch.nn.functional.binary_cross_entropy_with_logits(input = pred[...,-4:] , target = motion_[...,-4:])

tensor(0.5878, device='cuda:0')

In [30]:
ll = torch.nn.BCEWithLogitsLoss()

In [31]:
ll(input = pred[...,-4:] , target = motion_[...,-4:])

tensor(0.5878, device='cuda:0')

In [18]:
from utils.motion_processing.skeleton import Skeleton, t2m_kinematic_chain, t2m_raw_offsets


SyntaxError: invalid syntax (3771189632.py, line 2)

In [78]:
n_raw_offsets = torch.from_numpy(t2m_raw_offsets)
kinematic_chain = t2m_kinematic_chain


In [85]:
pwd

'/coc/scratch/sanisetty3/music_motion/ACMG'

In [19]:
%cd /srv/hays-lab/scratch/sanisetty3/smplx2hml

/coc/scratch/sanisetty3/smplx2hml


/srv/hays-lab/flash5/sanisetty3/miniconda3/envs/tgm3d/lib/python3.9/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [20]:
from motion_representation import motionx2positions, body_joints_id, hand_joints_id, n_raw_offsets, kinematic_chain
from paramUtil import t2m_kinematic_chain, t2m_raw_body_offsets
from common.skeleton import Skeleton

In [21]:
t2m_raw_body_offsets.shape

(22, 3)

In [24]:
body_joints_id

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21]

In [22]:
example_data = np.array(
        motionx2positions(
            "/srv/hays-lab/scratch/sanisetty3/motionx/motion_data/smplx_322/humanml/000021.npy"
        )
    )
example_data = example_data.reshape(len(example_data), -1, 3)
example_data = torch.from_numpy(example_data)[:, body_joints_id, :]

tgt_skel = Skeleton(torch.Tensor(t2m_raw_body_offsets), t2m_kinematic_chain, "cpu")
tgt_offsets = tgt_skel.get_offsets_joints(example_data[0])

In [112]:
np.load("/srv/hays-lab/scratch/sanisetty3/music_motion/ACMG/core/datasets/data/000021_pos.npy").shape

(270, 52, 3)

In [25]:
# np.save("/srv/hays-lab/scratch/sanisetty3/music_motion/ACMG/core/datasets/000021_pos.npy" , example_data)

In [49]:
mot = np.load("/srv/hays-lab/scratch/sanisetty3/motionx/motion_data/new_joint_vecs/idea400/subset_0000/Row_A_Boat_And_Walking_At_The_Same_Time.npy")
# xyz = np.array(recover_from_ric(torch.Tensor(mot) , 22))
# plot_3d.render(xyz , f"/srv/hays-lab/scratch/sanisetty3/music_motion/ACMG/render/og.gif")


In [48]:
mot[: , [1,2,3]] = 0


In [ ]:
xyz = np.array(recover_from_ric(torch.Tensor(mot) , 22))
plot_3d.render(xyz , f"/srv/hays-lab/scratch/sanisetty3/music_motion/ACMG/render/trans0.gif")


In [46]:
xyz = np.array(recover_from_rot(torch.Tensor(mot) , 22 , tgt_skel))
plot_3d.render(xyz , f"/srv/hays-lab/scratch/sanisetty3/music_motion/ACMG/render/trans0rot.gif")


In [47]:
mot.shape

(297, 623)

In [77]:
joint_num = 52
body_joints=22
hand_joints=30

In [55]:
aa = np.cumsum([4 , (joint_num - 1)*3 , (joint_num - 1)*6 , joint_num*3 , 4  ])

In [67]:
x = np.arange(8.0)
np.split(x, [3, 5, 6, 8])

[array([0., 1., 2.]),
 array([3., 4.]),
 array([5.]),
 array([6., 7.]),
 array([], dtype=float64)]

In [68]:
global_params , pos_data , rot_data , vel_data , foot_contact = np.split(mot , aa , -1)[:-1]

In [85]:
isinstance(pos_data , np.ndarray)

True

In [81]:
body_pos , hand_pos = np.split(pos_data , np.cumsum([(body_joints-1)*3 , hand_joints*3]) , -1)[:-1]
body_rot , hand_rot = np.split(rot_data , np.cumsum([(body_joints-1)*6 , hand_joints*6]) , -1)[:-1]
body_vel , hand_vel = np.split(vel_data , np.cumsum([(body_joints)*3 , hand_joints*3]) , -1)[:-1]

In [24]:
from enum import Enum

class MotionRep(Enum):
    FULL = "full"
    BODY = "body"
    HAND = "hand"
    LEFT_HAND = "left_hand"
    RIGHT_HAND = "right_hand"
    

In [33]:
MotionRep.FULL.value

'full'

In [17]:
from dataclasses import dataclass

In [20]:
@dataclass
class Motion:
    motion_rep = "full"
    hml_rep: str = "gprvc"
    root_params= None

In [21]:
mot = Motion()

In [ ]:
mot.getat

In [ ]:
vis(torch.Tensor(body_params) , dset , name = "og_motion" , joints = 22)

In [ ]:
np.zeros()

In [23]:

# body_params = dset.datasets[0].transform(body_params)

(293, 263)

In [17]:
motion = inputs["motion"][0].clone()
pred_motion = inputs["motion"][0].clone()
pred_motion[..., :4] = pred_translation

NameError: name 'pred_translation' is not defined

In [21]:
mask = inputs["motion"][1]
mask.shape

torch.Size([4, 116])

In [23]:
pred_motion.shape

torch.Size([4, 116, 263])

In [23]:
mott = next(iter(dset.datasets[0]))["motion"]
# mott2 = next(next(iter(dset.datasets[0])))["motion"]


In [32]:
inp, cnd = condition_provider(
            raw_motion=[mott , mott],
        )

True


In [37]:
inp["motion"][0].shape

torch.Size([2, 116, 263])

In [45]:
vis(torch.Tensor(motion[0].detach().cpu()) , dset , name = "og_motion" , joints = 22)

In [46]:
vis(torch.Tensor(pred_motion[0][:sum(mask[0])].detach().cpu()) , dset , name = "pred_motion" , joints = 22)

In [46]:
vis(torch.Tensor(body_params) , dset , name = "og_og_motion" , joints = 22)

In [51]:
mott.shape

(116, 263)

In [52]:
vis(torch.Tensor(mott) , dset , name = "dset_motion" , joints = 22)

In [82]:

sap = AttentionParams(dim = 256 , causal=True)
cap = AttentionParams(dim = 256 , causal=True , add_null_kv=True)
transformer_params = TranslationTransformerParams(self_attention_params = sap , 
                                                  cross_attention_params = cap , 
                                                  depth = 1, 
                                                  positional_embedding_params=PositionalEmbeddingParams(dim = 256) , 
                                                  positional_embedding=PositionalEmbeddingType.SINE,
                                                  fuse_method = {"cross_seperate" : ["audio" , "text"]}
                                                 )

In [62]:
from core.models.generation.translation_transformer import ClassifierFreeGuidanceDropout, TransformerBlock

In [63]:
dim = 256

In [64]:
pos_emb = ScaledSinusoidalEmbedding(dim , theta = 10000)
cfg_dropout = ClassifierFreeGuidanceDropout(
            0.0
        )
condition_fuser = ConditionFuser({"cross" : ["audio"] ,   "prepend" : ["text"]})

In [65]:
transformer_blocks = TransformerBlock(sap , cap).cuda()

In [66]:
project_audio = (
    nn.Linear(128, dim).cuda()
    
)
project_text = (
    nn.Linear(768, dim).cuda()
    
)

In [67]:
motion = inputs["motion"][0]
motion_padding_mask = inputs["motion"][1]
device, b, n , d = motion.device, *motion.shape
translation = motion[... , :4]

In [68]:
x = (
    pos_emb(translation).repeat(b, 1, 1) * motion_padding_mask.unsqueeze(-1),
    motion_padding_mask,
)


In [69]:
conditions = cfg_dropout(conditions , 0.2)


In [70]:
audio_embed = project_audio(conditions["audio"][0])
text_embed = project_text(conditions["text"][0])


In [71]:
conditions["audio"] = (audio_embed, conditions["audio"][1])
conditions["text"] = (text_embed, conditions["text"][1])


In [72]:
conditions["audio"][0].shape

torch.Size([4, 1, 256])

In [75]:
inputs_, cross_inputs_ = condition_fuser(x, conditions)

In [78]:
x_ = inputs_[0]
x_padding_mask = inputs_[1]
context = cross_inputs_[0]
context_padding_mask = cross_inputs_[1]

In [81]:
x_.shape

torch.Size([4, 153, 256])

In [82]:
x_padding_mask.shape

torch.Size([4, 153])

In [83]:
context.shape

torch.Size([4, 1, 256])

In [84]:
context_padding_mask.shape

torch.Size([4, 1])

In [85]:
conditions["text"][1]

tensor([[ True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False],
        [False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False],
        [ True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False],
        [ True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  Tr

In [86]:
x_padding_mask

tensor([[ True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  

In [87]:
embed = transformer_blocks(
            x=x_,
            mask=x_padding_mask,
            context=context,
            context_mask=context_padding_mask,
        )

In [89]:
embed = embed[:, -n:, :]

In [90]:
embed.shape

torch.Size([4, 116, 256])

## Translation

In [120]:
from core.models.attend import Attention
from core.models.dataclasses import AttentionParams, TransformerParams, PositionalEmbeddingParams, PositionalEmbeddingType
from core.models.utils import get_obj_from_str


In [117]:
from core.models.dataclasses import AttentionParams, TransformerParams, PositionalEmbeddingParams, PositionalEmbeddingType

sap = AttentionParams(dim = 256 , causal=True)
cap = AttentionParams(dim = 256 , causal=True , add_null_kv=True)
transformer_params = TransformerParams(attention_params = sap , cross_attention_params = cap , positional_embedding_params=PositionalEmbeddingParams(dim = 256) , positional_embedding=PositionalEmbeddingType.SINE )
transformer_params.depth = 4
transformer_params.dim_out = 4
transformer_params.audio_input_dim = 128
transformer_params.text_input_dim = 768

In [174]:
project_audio = nn.Linear(128 , 256).cuda()
project_text = nn.Linear(768 , 256).cuda()
aud = project_audio(conditions["audio"][0])
txt = project_text(conditions["text"][0])
conditions["audio"] = (aud , conditions["audio"][1])
conditions["text"] = (txt , conditions["text"][1])

In [143]:
pos_emb = ScaledSinusoidalEmbedding(PositionalEmbeddingParams(dim = 256))

In [161]:
b , n , _ = input["motion"][0].shape

In [167]:

x = pos_emb(input["motion"][0]).repeat(b , 1 ,1)


In [173]:
x.shape

torch.Size([4, 116, 256])

In [ ]:
for a, (b , c) in conditions.items():
    print(a)
    print(b.shape)

In [175]:
inputs_ , cross_inputs = condition_fuser(x , conditions  )

In [176]:
inputs_.shape

torch.Size([4, 153, 256])

In [177]:
cross_inputs.shape

torch.Size([4, 500, 256])